In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from AMPpred_MFA.lib.Encoding import easy_encoding
from AMPpred_MFA.lib.Vocab import load_vocab
from AMPpred_MFA.lib.Data import load_dataset, get_data_iter, get_data_loader
from AMPpred_MFA.models.Model import load_model
from AMPpred_MFA.models.AMPpred_MFA import Model, Config
import matplotlib as mpl

mpl.rcParams['font.family'] = 'Times New Roman'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
save_path_attention_visual = '../figures/attention visualization/A0A1P8AQ95' # Saving path
os.makedirs(save_path_attention_visual, exist_ok=True)

In [2]:
vocab_path = './trained_model/vocab.json'
model_path = './trained_model/model.pth'
vocab = load_vocab(vocab_path)
config = Config()
config.k_mer = 1
config.batch_size = 32
config.embed_padding_idx = vocab[config.padding_token]
config.feature_dim = 400
config.vocab_size = len(vocab)
model = Model(config)
load_model(model, model_path)
model.eval()
fastas = np.array([
    ('A0A1P8AQ95','MTKNMTKKKMGLMSPNIAAFVLPMLLVLFTISSQVEVVESTGRKLSWAFNGAPIVFTPPSSSCGGSPAAVMASEWMPRRPCRRTRPPGTNIPVSQSP'),
    ])
data_x = easy_encoding(fastas, 'mixed', vocab,
                        config.k_mer, config.padding_size)

In [3]:
out = model(data_x)
attention2 = model.attention_wight2.cpu().detach().numpy()
attention2_inputs = model.attention_wight2_inputs.cpu().detach().numpy().reshape(config.padding_size, -1)
attention2_outputs = model.attention_wight2_outputs.cpu().detach().numpy().reshape(config.padding_size, -1)
print(attention2.shape)
print(attention2_inputs.shape)
print(attention2_outputs.shape)

In [4]:
def show_attention_heatmap(attention_weights: np.ndarray):
    """
    Heatmap visualization of multi-head attention weight matrix
     'viridis': blue-white-yellow, gradually darkening
     'plasma': purple-yellow, progressively darker
     'inferno': black, red, yellow and white, progressively darker
    Args:
        attention_weights: np.ndarray, Attention weight matrix, shape is (num_heads, seq_length, seq_length)
    """ 
    num_heads, seq_length, _ = attention_weights.shape
    # Calculate the number of images to be displayed and their arrangement, try to display them in a square
    num_cols = int(np.ceil(np.sqrt(num_heads)))
    num_rows = int(np.ceil(num_heads / num_cols))
    # Create a subgraph of num_rows * num_cols
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(num_cols * 8, num_rows * 8))
    # Flatten the axis to facilitate subsequent traversal
    axs = axs.flatten()
    for i, ax in enumerate(axs):
        head_attention = attention_weights[i, :, :]
        im = ax.imshow(head_attention, cmap='viridis')
        ax.set_xticks(range(0, seq_length, 3),range(0 ,seq_length, 3), fontsize=14,rotation=90)
        ax.set_yticks(range(0, seq_length, 3),range(0 ,seq_length, 3), fontsize=14)
        ax.tick_params(axis='both')
        ax.set_xlabel('Output position')
        ax.set_ylabel('Input position')
        ax.set_title('Attention head {}'.format(i+1))
        # Add color bar
        cbar = ax.figure.colorbar(im, ax=ax, shrink=0.7)
        # cbar.ax.set_ylabel('Attention weight', va='center', labelpad=10, rotation=-90)
    # Delete unused subgraphs
    for i in range(num_rows * num_cols - num_heads):
        fig.delaxes(axs[-1,-(i+1)])
    # fig.tight_layout()
    fig.subplots_adjust(left=0, bottom=0.09, right=0.9, top=0.9, wspace=0.16, hspace=0)
    return fig

mpl.rcParams.update({'font.size': 20})  
fig_attention_heatmap = show_attention_heatmap(attention2)


In [5]:
def show_attention_feature_ranking(attention_weights: np.ndarray, num_features=20):
    """
    Feature Importance Ranking for Attention Weights
    
    Args:
        attention_weights: np.ndarray, Attention weight matrix, the shape is (num_heads, seq_length, seq_length)
        num_features: int, Show first num_features features
    """ 
    # Averaging the multi-head attention weight matrix
    avg_weights = np.mean(attention_weights, axis=0)
    # Calculate the output attention value of the current position (sum the input dimension to get the attention value of the output dimension)
    output_weights = np.sum(avg_weights, axis=0)
    # Sort features in descending order by attention value
    sorted_idx = np.argsort(output_weights)[::-1][:num_features]
    sorted_features = output_weights[sorted_idx]
    # Draw histogram
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.bar(np.arange(num_features), sorted_features)
    ax.set_xticks(np.arange(num_features),[i for i in sorted_idx])
    # ax.spines['right'].set_visible(False)
    # ax.spines['top'].set_visible(False)
    ax.tick_params(axis='both')
    ax.set_xlabel('Position')
    ax.set_ylabel('Attention value')
    ax.set_title('Top {} of attention values'.format(num_features))
    plt.tight_layout()
    return fig
mpl.rcParams.update({'font.size': 12})  
fig_feature_ranking = show_attention_feature_ranking(attention2, 30)

In [6]:
def show_attention_distribution(attention_weights: np.ndarray):
    """
    Boxplots for visualizing the output of multi-head attention
    
    Args:
    attention_weights: numpy array, multi-head attention weight matrix, shape is (num_heads, seq_length, seq_length)
    """
    num_heads, seq_length, _ = attention_weights.shape
    # Sum over the input dimension to get the attention of the output position under different heads
    output_weights = attention_weights.sum(axis=1)
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs = axs.flatten()
    fig.suptitle('Output feature distribution')
    # Boxplot
    sns.boxplot(data=output_weights.T, ax=axs[0])
    axs[0].set_title('Head-wise attention distribution')
    axs[0].set_xticks(np.arange(0,4), np.arange(1,5))
    axs[0].set_xlabel('Attention head')
    axs[0].set_ylabel('Attention value')
    # Barplot
    axs[1].bar(np.arange(seq_length), output_weights.mean(axis=0))
    axs[1].set_title('Position-wise attention distribution')
    axs[1].set_xlabel('Postion')
    axs[1].set_ylabel('Attention value')
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.2)
    return fig

mpl.rcParams.update({'font.size': 14})  
fig_output = show_attention_distribution(attention2)

In [7]:
def show_attention_input_output(inputs: np.ndarray,outputs: np.ndarray):
    """
    Visualizing the output matrix and output matrix of multi-head attention
    
    Args:
    inputs: numpy array, the input matrix of the multi-head attention layer, shape is (seq_length, embedding_size)
    outputs: numpy array, the output matrix of the multi-head attention layer, the shape is (seq_length, embedding_size)
    """
    seq_length, embedding_size = inputs.shape
    fig, axs = plt.subplots(2, 1, figsize=(6, 10))
    axs = axs.flatten()
    # Plot a heatmap of the input matrix
    sns.heatmap(inputs.T, cmap='viridis', ax=axs[0])
    axs[0].set_title('Original features before processing by the attention layer')
    axs[0].set_xlabel('Position')
    axs[0].set_ylabel('Embedding')
    axs[0].set_xticks(range(0, seq_length, 5),range(0, seq_length, 5), fontsize=10)
    axs[0].set_yticks(range(0, embedding_size, 5),range(0, embedding_size, 5), fontsize=10)
    # Plot a heatmap of the output matrix
    sns.heatmap(outputs.T, cmap='viridis', ax=axs[1])
    axs[1].set_title('Implicated features after processing by the attention layer')
    axs[1].set_xlabel('Position')
    axs[1].set_ylabel('Embedding')
    axs[1].set_xticks(range(0, seq_length, 5),range(0, seq_length, 5), fontsize=10)
    axs[1].set_yticks(range(0, embedding_size, 5),range(0, embedding_size, 5), fontsize=10)
    fig.tight_layout()
    return fig

mpl.rcParams.update({'font.size': 14})  
fig_input_output_features = show_attention_input_output(attention2_inputs, attention2_outputs)

In [8]:
import random
def normalize(values):
    """
    Normalize the incoming list
    """
    min_val = min(values)  
    max_val = max(values) 
    if min_val == max_val: 
        return [1.0] * len(values)
    else:
        return [(val - min_val) / (max_val - min_val) for val in values]  

def generate_random_color():
    """
    Generate random rgb tuples
    """
    r, g, b = np.random.randint(0,255,size=3)
    # Limit luma brightness value above 70
    while 0.2126 * r + 0.7152 * g + 0.0722 * b > 70:
        r, g, b = np.random.randint(0,255,size=3)
    return (r/255, g/255, b/255) 

def show_attention_motifs(attention_weights, sequence, k_mer=1):
    """
    Visual attention motifs
    
    Args:
    attention_weights: numpy array, multi-head attention weight matrix, shape is (num_heads, seq_length, seq_length)
    sequence: str, the protein sequence
    k_mer: int, k value of k_mer
    """
    # Compute the attention score for each location
    avg_attention_weights = attention_weights.mean(axis=0)
    attention_scores = avg_attention_weights.sum(axis=0)
    attention_scores = normalize(attention_scores)
    # Divide the sequence into several k-mer fragments
    motif_names = [sequence[i:i+k_mer] for i in range(len(sequence)-k_mer-1)]
    num_motifs = len(motif_names)
    # Calculate the height at which each motif should be displayed
    motif_size = attention_scores / np.max(attention_scores) * 100
    # Initialize image
    fig, ax = plt.subplots(figsize=(20,4)) 
    color_dict = {}
    for i in range(num_motifs):
        # Motif vertical formatting
        motif = '\n'.join(list(motif_names[i]))
        if attention_scores[i] in color_dict:
            color = color_dict[attention_scores[i]]
        else:
            color =generate_random_color()
            color_dict[attention_scores[i]] = color
        ax.text(i, 0, motif, ha='center', va='bottom',  fontsize=motif_size[i], color=color)
    ax.set_xlim(0, num_motifs)
    ax.set_ylim(0, k_mer)
    ax.axis('off')
    return fig


fig_motif = show_attention_motifs(attention2, fastas[0][1])

In [9]:
def show_attention_network(attention_weights: np.ndarray):
    """
    Attention Network Visualization
    Args:
        attention_weights: np.ndarray, Attention weight matrix, the shape is (num_heads, seq_length, seq_length)
    """ 
    num_heads, seq_length, _ = attention_weights.shape
    fig, axs = plt.subplots(1, 4, figsize=(6*4, 10),facecolor='black')

    fig.suptitle('Network of attention')
    colors = ['r','g','b','orange']
    for i, ax in enumerate(axs):
        head_attention = attention_weights[i, :, :]
        # Calculate the maximum attention index for each output position
        attention = head_attention.argmax(axis=1)
        unique_elements, counts = np.unique(attention, return_counts=True)
        # Set line transparency
        freq_dict = dict(zip(unique_elements, counts/max(counts)))
        for y,idx in enumerate(attention):
            ax.plot([1,4],[98-y,idx],alpha=freq_dict[idx],color=colors[i],linewidth=2)
        ax.set_title('Attention head {}'.format(i+1), color = 'white')
        ax.set_xlim(0, 5)
        ax.set_ylim(0, 100)
        ax.axis('off')    
    plt.tight_layout()
    return fig

mpl.rcParams.update({'font.size': 28})  
fig_attention_network = show_attention_network(attention2)

In [10]:
fig_attention_heatmap.savefig(os.path.join(save_path_attention_visual, 'attention heatmap.png'), dpi=300, bbox_inches='tight')
fig_feature_ranking.savefig(os.path.join(save_path_attention_visual, 'feature ranking.png'), dpi=300, bbox_inches='tight')
fig_output.savefig(os.path.join(save_path_attention_visual, 'output feature distribution.png'), dpi=300, bbox_inches='tight')
fig_input_output_features.savefig(os.path.join(save_path_attention_visual, 'inputs and outputs features.png'), dpi=300, bbox_inches='tight')
fig_motif.savefig(os.path.join(save_path_attention_visual, 'motif.png'), dpi=300, bbox_inches='tight')
fig_attention_network.savefig(os.path.join(save_path_attention_visual, 'attention network.png'), dpi=300, bbox_inches='tight')
